### Installing the required Modules


In [1]:
# Importing the require modules
import numpy as np

import os
import pickle 
import blosc

import re
from pprint import pprint

from openbabel import pybel
from openbabel import openbabel as ob
from xtb.interface import Calculator, Param
from xtb.libxtb import VERBOSITY_MUTED

error_files = []

## Helper function filter to extract all the bond lengths, angles and dihedrals from a molecule

In [2]:
def filterAtom(atom: str) -> str:
    '''Removes any number from the string e.g. C3 becomes C'''
    pattern_order = r'[0-9]'
    return re.sub(pattern_order, '', atom)

def atomType(mol, atomIdx) -> str:
    '''get the atomic type given an atom index'''
    return mol.OBMol.GetAtom(atomIdx).GetType()

In [3]:
def getBondOrders_Charges(mol):
    
    coords = []
    for atom in mol.atoms:
        coords.append([atom.coords[0], atom.coords[1], atom.coords[2]])
    
    # xtb-python expects them in Bohr
    atom_coords = np.array(coords) / 0.52917721092
    atomic_nums = np.array([a.atomicnum for a in mol.atoms])
    
    # Creating the calculator
    calc = Calculator(Param.GFN2xTB, atomic_nums, atom_coords)
    calc.set_verbosity(VERBOSITY_MUTED)
    res = calc.singlepoint()
    
    # Get the partial charges
    charges = res.get_charges()
    
    # Get the bond order matrix
    bond_orders = res.get_bond_orders()
    
    '''
    print("Charges:")
    for i, charge in enumerate(charges):
        print("%d\t%f" % (i+1, charge))
        
    print("Bond orders (matrix):")
    for i, row in enumerate(bond_orders):
        for j, bond_order in enumerate(row):
            if j > i and bond_order > 0.1:
                print("%d \t %d \t %f" % (i+1, j+1, bond_order))
    '''
    
    return (charges, bond_orders)

In [11]:
def NeighboursList_Prop(b, a, c, mol, charge, bond_order):
    neighbours_list = []
    for neighbor in ob.OBAtomAtomIter(b):
        if (neighbor.GetIdx() != a.GetIdx() and neighbor.GetIdx() != c.GetIdx()):
            neighbours_list.append(neighbor.GetIdx());
            
    neighbour_prop = []
    # Dictionary of atom, bond_order, charge, ringsize
    for n in neighbours_list:
        try:
            neighbour_prop.append([filterAtom(atomType(mol, n)), charge[n-1], bond_order[n-1][b.GetIdx()-1], 
                            mol.OBMol.GetAtom(n).MemberOfRingSize()])
        except: 
            print(n, b.GetIdx(), bond_order, "\n")
            pass
    return neighbour_prop

In [5]:
def getBonds(mol, charge, bond_order, filename) -> dict:
    '''Iterate through all the bonds in a molecule'''
    
    bonds = {}
    
    for bond in ob.OBMolBondIter(mol.OBMol):
        index1 = bond.GetBeginAtomIdx()
        index2 = bond.GetEndAtomIdx()
        begin = filterAtom(atomType(mol, index1))
        end = filterAtom(atomType(mol, index2))
        bond_length = round(bond.GetLength(),4)
        bondOrder = bond_order[index1-1][index2-1]
        
        # Swap them for lexographic order
        if (end < begin):
            begin, end = end, begin
        
        #Appending to the dictionary of list
        if (f"{begin} - {end} , {bond_order}" in bonds.keys()):
            bonds[f"{begin} - {end} , {round(bondOrder,1)}"].append((bond_length, filename, charge[index1-1], charge[index2-1], 
                                                             round(bondOrder,2)))
        else:
            bonds[f"{begin} - {end} , {round(bondOrder,1)}"]=[(bond_length, filename, charge[index1-1], charge[index2-1], 
                                                             round(bondOrder,2))]
        
    return bonds

In [6]:
def getAngles(mol, bondOrder, charge, filename) -> dict:
    '''Iterate through all the bond angles in a molecule'''
    
    angles = {}
    
    for angle in ob.OBMolAngleIter(mol.OBMol):
        bond_order1 = bondOrder[angle[0], angle[1]]
        bond_order2 = bondOrder[angle[0], angle[2]]
        #print(bond_order1, bond_order2)
        #print(angle[0], angle[1], angle[2])
        
        b = mol.OBMol.GetAtom(angle[0] + 1)
        a = mol.OBMol.GetAtom(angle[1] + 1)
        c = mol.OBMol.GetAtom(angle[2] + 1)
        bond_angle = mol.OBMol.GetAngle(a, b, c)
        
        # check for rings .. if any of the atoms are 
        aType = filterAtom(a.GetType())
        bType = filterAtom(b.GetType())
        cType = filterAtom(c.GetType())
        
        aRingSize = a.MemberOfRingSize()
        bRingSize = b.MemberOfRingSize()
        cRingSize = c.MemberOfRingSize()
        
        # Hybridization
        b_hybrid = b.GetHyb()
        
        # Getting neighbour
        neighbour = NeighboursList_Prop(b, a, c, mol, charge, bond_order)
        
        # Swap them for lexographic order
        if (cType < aType):
            aType, cType = cType, aType
            bond_order1, bond_order2 = bond_order2, bond_order1
            aRingSize, cRingSize = cRingSize, aRingSize
            
        #Appedning to the dictonary of lists if the angle b/t the elements already existse
        if(f"{aType} - {bType} - {cType}" in angles.keys()):
            angles[f"{aType} - {bType} - {cType}"].append((bond_angle, filename, charge[angle[0]], charge[angle[1]], charge[angle[2]], 
                                                           bond_order1, bond_order2, b_hybrid, aRingSize, bRingSize, cRingSize, neighbour))
        else:
            angles[f"{aType} - {bType} - {cType}"] = [(bond_angle, filename, charge[angle[0]], charge[angle[1]], charge[angle[2]], 
                                                       bond_order1, bond_order2, b_hybrid, aRingSize, bRingSize, cRingSize, neighbour)]                                                                                      
                                                                                                     
    return angles

In [7]:
def getTorsions(mol, bondOrder, charge, filename) -> dict:
    '''Iterate through all the torsions in a molecule'''
    
    torsions = {}
    
    for torsion in ob.OBMolTorsionIter(mol.OBMol):
        a = torsion[0] + 1
        b = torsion[1] + 1
        c = torsion[2] + 1
        d = torsion[3] + 1
        
        bondOrder1 = bondOrder[torsion[0], torsion[1]]
        bondOrder2 = bondOrder[torsion[1], torsion[2]]
        bondOrder3 = bondOrder[torsion[2], torsion[3]]
        
        torsion_angle = round(mol.OBMol.GetTorsion(a, b, c, d), 3)

        aType = filterAtom(atomType(mol, a))
        bType = filterAtom(atomType(mol, b))
        cType = filterAtom(atomType(mol, c))
        dType = filterAtom(atomType(mol, d))

        # Switch if not in lexographic order
        if(dType < aType):
            aType, dType = dType, aType
            bondOrder1, bondOrder3 = bondOrder3, bondOrder1
            
        #Appedning to the dictonary of lists if the torsion angle b/t the elements already exists
        if(f"{aType} - {bType} - {cType} -{dType}" in torsions.keys()):
            torsions[f"{aType} - {bType} - {cType} -{dType}"].append((torsion_angle, filename, charge[torsion[0]], charge[torsion[1]],
                                                charge[torsion[2]], charge[torsion[3]], bondOrder1, bondOrder2, bondOrder3))
        
        # Checking for palindromic sequence
        elif(f"{dType} - {cType} - {bType} -{aType}" in torsions.keys()):
            torsions[f"{dType} - {cType} - {bType} -{aType}"].append((torsion_angle, filename, charge[torsion[3]], charge[torsion[2]],
                                                charge[torsion[1]], charge[torsion[0]], bondOrder3, bondOrder2, bondOrder1))
            
        else:
            torsions[f"{aType} - {bType} - {cType} -{dType}"] = [(torsion_angle, filename, charge[torsion[0]], charge[torsion[1]],
                                            charge[torsion[2]], charge[torsion[3]], bondOrder1, bondOrder2, bondOrder3)]
    
    return torsions

### Wrapping all the helper functions into a main function

In [8]:
def analyze_molecular_data(file: str) -> (dict, dict, dict):
    '''Fetches the bond lengths, angles and dihedrals from a given file'''
    
    # Load the File
    extension=file.split('.')[-1]
    try:
        mol = next(pybel.readfile(extension, file))
    except:
        error_files.append(file)
        return ({},{},{})
    
    # Get Bond Order and partial charges on atoms
    charge, bond_order = getBondOrders_Charges(mol)
    
    # Get bond lengths
    bond_lengths = getBonds(mol, charge, bond_order, file)
    
    # Get bond angles
    bond_angles = getAngles(mol, bond_order, charge, file)
    
    # Get torsions
    dihedrals = getTorsions(mol, bond_order, charge, file)

    return (bond_lengths, bond_angles, dihedrals)


def mergeDictionaries(dict1 : dict, dict2: dict):
    '''Merges two Python dictionary by combining elements for a common key
    The function merges dict2 into dict 1'''
    for key, value in dict2.items():
        if key in dict1:
            dict1[key].extend(value)
        else:
            dict1[key] = value

### Testing the above functions on a sample molecule

In [ ]:
# Example usage
xyz_file = '21395061/cod-crest/W/WKGCLMYDEAXJHN-UHFFFAOYSA-N.xyz'
mol = next(pybel.readfile('xyz', xyz_file))
charge, bond_order = getBondOrders_Charges(mol)
#pprint(getBonds(mol))
getAngles(mol, bond_order, charge, xyz_file)
#getTorsions(mol, bond_order, charge)


#molecular_data = analyze_molecular_data(xyz_file)
#molecular_data

## Extracting Molecular Data from all the files

In [ ]:
#Collecting data from the Molecular Dataset

bond_lengths = {}
bond_angles = {}
dihedrals = {}
filenames = []

ext = ('.xyz',)
count = 0

for root, dirs, files in os.walk('21395061/cod-crest/'):
    for file in files:
        extension=os.path.splitext(file)[1]
        count +=1
        if(extension not in ext): continue
            
        filename = os.path.join(root,file)
        filenames.append(filename)
        length, angle, torsion = analyze_molecular_data(filename)
        
        mergeDictionaries(bond_lengths, length)
        mergeDictionaries(bond_angles, angle)
        mergeDictionaries(dihedrals, torsion)

### Running the corrupt xyz files with corresponding sdf files and writing the bond length, angle, torsion data as pickle object

In [ ]:
filenames[-1]

In [ ]:
#Trying with sfd files for the corrupted xyz files

for file in error_files:
    filename = os.path.splitext(file)[0] + '.sdf'
    length, angle, torsion = analyze_molecular_data(filename)
    #mergeDictionaries(bond_lengths, length)
    mergeDictionaries(bond_angles, angle)
    mergeDictionaries(dihedrals, torsion)

In [ ]:
error_files

In [ ]:
def MolecularDataWriter_compress():
    
    pickled_bond_lengths = pickle.dump(bond_lengths)
    compressed_bond_lengths = blosc.compress(pickled_bond_lengths)
    
    with open('cod-crest_bond_lengths.dat', 'wb') as f:
        f.write(compressed_bond_lengths)
        
    pickled_bond_angles = pickle.dump(bond_angles)
    compressed_bond_angles = blosc.compress(pickled_bond_angles)
        
    with open('cod-crest_bond_angles.dat', 'wb') as f:
        f.write(compressed_bond_angles)
        
    pickled_torsions = pickle.dump(torsion)
    compressed_dihedrals = blosc.compress(pickled_torsions)
        
    with open('cod-crest_torsions.dat', 'wb') as f:
        f.write(compressed_dihedrals)

In [ ]:
def MolecularDataWriter_pickle():
    
    for key in bond_lengths.keys():
        bond_lengths[key] = np.array(bond_lengths[key])
        
    for key in bond_angles.keys():
        bond_angles[key] = np.array(bond_angles[key])
        
    for key in dihedrals.keys():
        dihedrals[key] = np.array(dihedrals[key])
    
#     with open('cod-crest_bond_lengths_.pkl', 'wb') as f:
#         pickle.dump(bond_lengths, f)
        
    with open('cod-crest_bond_angles.pkl', 'wb') as f:
        pickle.dump(bond_angles, f)
        
    with open('cod-crest_torsions.pkl', 'wb') as f:
        pickle.dump(dihedrals, f)

In [ ]:
for key in bond_angles.keys():
    print(key)

In [ ]:
MolecularDataWriter_pickle()